<a href="https://colab.research.google.com/github/ariflaksito/recsys/blob/master/02_cb_hotelbdgdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-based Filtering
Hotel Bandung DB<br />
Data dan code diambil dari https://medium.com/@nprasetio

# Import library

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import re
import random

# Import data

In [24]:
hotels = pd.read_csv("/content/drive/My Drive/Colab Data/hotel_bandung_english.csv")
hotels.head()

,name,address,description
0,Capital O 253 Topas Galeria Hotel,"Jl. Dr. Djundjunan No. 153, 40173 Bandung, Ind...","A 10-minute drive from Bandung Airport, Topas ..."
1,Sheraton Bandung Hotel & Towers,"Jl. Ir H Juanda 390, 40135 Bandung, Indonesia",Sheraton Hotel & Towers offers 5-star accommod...
2,OYO 794 Ln 9 Bandung Residence,"Jalan Lemahnendeut No 9, Sukajadi, 40164 Bandu...","Conveniently located in Sukajadi, Bandung, OYO..."
3,OYO 226 LJ hotel,"Jl. Malabar No.2, Malabar, Lengkong, Dago, Asi...","Featuring a shared lounge, OYO 226 LJ hotel is..."
4,OYO 230 Maleo Residence,"JI. Dangeur Indah II No. 15, Sukagalih, Sukaja...",Attractively set in the Sukajadi district of B...


In [25]:
hotels.describe()

,name,address,description
count,105,105,105
unique,101,98,103
top,OYO 794 Ln 9 Bandung Residence,"Jalan Lemahnendeut No 9, Sukajadi, 40164 Bandu...","Conveniently located in Sukajadi, Bandung, OYO..."
freq,3,3,2


In [26]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         105 non-null    object
 1   address      105 non-null    object
 2   description  105 non-null    object
dtypes: object(3)
memory usage: 2.6+ KB


# Fungsi deskripsi item hotel

In [0]:
def print_description(index):
    example = hotels[hotels.index == index][['description', 'name', 'address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Alamat:', example[2])  

In [28]:
print_description(5)

Offering free Wi-Fi access throughout the hotel, Dago's Hill Hotel, Business and Leisure is a 10-minute drive from Dago area which offers a variety of factory outlets. The hotel has an outdoor pool and modern rooms with private bathrooms. Paris Van Java Shopping Mall is a 20-minute drive from Dago's Hill Hotel.Husein Sastranegara Airport and Bandung Central Station can be reached in a 30-minute drive. It takes 45 minutes to drive to Trans Studio Indoor Amusement Park from this property. All well-equipped rooms offer tiled floors and air conditioning. Each room has a desk, telephone and flat-screen cable TV. Safes in certain rooms are available. The private bathroom provides shower facilities. A variety of facilities include laundry services and meeting / banquet facilities. Staff at the 24-hour front desk can also arrange car rentals and airport transfers. Indonesian and international dishes are served at the hotel’s restaurant. You can also enjoy meals in the comfort of your room.
Nam

# Text processing

In [0]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

# Tambah field baru
Atribut hotel ditambahkan description dengan data yang telah dibersihakan dari fungsi clean_text() diatas

In [30]:
  hotels['desc_clean'] = hotels['description'].apply(clean_text)
  hotels.head()

,name,address,description,desc_clean
0,Capital O 253 Topas Galeria Hotel,"Jl. Dr. Djundjunan No. 153, 40173 Bandung, Ind...","A 10-minute drive from Bandung Airport, Topas ...",10minute drive bandung airport topas galeria h...
1,Sheraton Bandung Hotel & Towers,"Jl. Ir H Juanda 390, 40135 Bandung, Indonesia",Sheraton Hotel & Towers offers 5-star accommod...,sheraton hotel towers offers 5star accommodati...
2,OYO 794 Ln 9 Bandung Residence,"Jalan Lemahnendeut No 9, Sukajadi, 40164 Bandu...","Conveniently located in Sukajadi, Bandung, OYO...",conveniently located sukajadi bandung oyo 794 ...
3,OYO 226 LJ hotel,"Jl. Malabar No.2, Malabar, Lengkong, Dago, Asi...","Featuring a shared lounge, OYO 226 LJ hotel is...",featuring shared lounge oyo 226 lj hotel locat...
4,OYO 230 Maleo Residence,"JI. Dangeur Indah II No. 15, Sukagalih, Sukaja...",Attractively set in the Sukajadi district of B...,attractively set sukajadi district bandung oyo...


# TF-IDF dan Cosine Similarity

In [31]:
print(hotels.columns)

Index(['name', 'address', 'description', 'desc_clean'], dtype='object')


In [32]:
hotels.set_index('name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(hotels['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.02618853, 0.01554192, ..., 0.01022334, 0.03967072,
        0.0349132 ],
       [0.02618853, 1.        , 0.0200914 , ..., 0.01765244, 0.03436038,
        0.0256194 ],
       [0.01554192, 0.0200914 , 1.        , ..., 0.15477556, 0.01379571,
        0.02738874],
       ...,
       [0.01022334, 0.01765244, 0.15477556, ..., 1.        , 0.01018232,
        0.02326799],
       [0.03967072, 0.03436038, 0.01379571, ..., 0.01018232, 1.        ,
        0.03714926],
       [0.0349132 , 0.0256194 , 0.02738874, ..., 0.02326799, 0.03714926,
        1.        ]])

In [33]:
indices = pd.Series(hotels.index)
indices[:15]

0                Capital O 253 Topas Galeria Hotel
1                  Sheraton Bandung Hotel & Towers
2                   OYO 794 Ln 9 Bandung Residence
3                                 OYO 226 LJ hotel
4                          OYO 230 Maleo Residence
5                        OYO 167 Dago's Hill Hotel
6                   OYO 794 Ln 9 Bandung Residence
7                       OYO 196 Horizone Residence
8     OYO 483 Flagship Tamansari Panoramic Bandung
9               OYO 295 Grha Ciumbuleuit Residence
10                            OYO 193 SM Residence
11              Capital O 874 Hotel Nyland Pasteur
12                            OYO 352 Sabang Hotel
13                                  Hilton Bandung
14             InterContinental Bandung Dago Pakar
Name: name, dtype: object

# Modelling

In [0]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_hotel = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_hotel.append(list(hotels.index)[i])
        
    return recommended_hotel

# Cek rekomendasi

In [38]:
recommendations('Serela Cihampelas Hotel')

['Grand Sovia Hotel',
 'Vio Cihampelas',
 'Neo Dipatiukur Bandung',
 'Grand Tjokro Bandung',
 'HARRIS Hotel & Conventions Ciumbuleuit - Bandung',
 'InterContinental Bandung Dago Pakar',
 'Ibis Bandung Pasteur',
 'Aryaduta Bandung',
 'Tebu Hotel Bandung',
 'Benua Hotel']